In [1]:
import folium
import webbrowser
from folium.plugins import MarkerCluster
import osmnx as ox
import geopandas as gpd

# Specify the location you want to focus on
place_name = "Nakuru, Kenya"

# Fetch OSM data for Nakuru
graph = ox.graph_from_place(place_name, network_type="drive")
nodes, edges = ox.graph_to_gdfs(graph)
amenities = ox.features_from_place(place_name, tags={"amenity": True})

# Convert amenities to GeoDataFrame
amenity_points = gpd.GeoDataFrame(amenities)

# Create a map centered on Nakuru, Kenya
nakuru_map = folium.Map(
    location=amenity_points.unary_union.centroid.coords[0][::-1],
    zoom_start=13,
    tiles="OpenStreetMap",
)  # Use OSM tiles

# Cluster markers for better visualization
marker_cluster = MarkerCluster().add_to(nakuru_map)

# Add amenity markers to the map with different colors
for idx, row in amenity_points.iterrows():
    # Check if geometry is a point or polygon
    if row.geometry.geom_type == "Point":
        location = [row.geometry.y, row.geometry.x]
    else:
        location = row.geometry.centroid.coords[0][::-1]  # Get centroid for polygons

    folium.Marker(
        location=location,
        popup=row["amenity"],
        icon=folium.Icon(color="blue", icon="info-sign"),  # Customize icons
    ).add_to(marker_cluster)

# Check if 'u' and 'v' columns are present in edges
if 'u' in edges.columns and 'v' in edges.columns:
    # Add roads from OpenStreetMap data if columns exist
    folium.GeoJson(
        edges[["u", "v", "geometry"]].to_json(),
        name="Roads",
        style_function=lambda x: {"color": "gray", "weight": 2},
    ).add_to(nakuru_map)
else:
    print("Warning: 'u' and 'v' columns not found in edges GeoDataFrame. Roads will not be displayed.")

# Save the map and open in browser
filename = "nakuru_amenities_map_osm.html"
nakuru_map.save(filename)
webbrowser.open(filename)  


True

[Click here to view the interactive map](nakuru_amenities_map_osm.html)
